In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

#### Perfrom some webscraping to get additional data for falcon 9 launches

In [2]:
# Some premade functions to process the web scraped HTML tables

def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    

In [11]:
# Load the url of a 2019 wikipedia page to a responce object, and create a beautiful soup object to parse the HTML content

static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

responce = requests.get(static_url)

html_content = responce.text

soup = BeautifulSoup(html_content, 'html.parser')

In [12]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [15]:
html_tables = soup.find_all('table')

In [24]:
# Want the tables which contain the launch records, from some trial and error, can see this is from the third table onwards

first_launch_table = html_tables[2]

In [75]:
# Make a list of column names by iterating through the <th> elements using the defined extract_column_from_header function

column_names = []

# Find all rows in the table
for row in first_launch_table.find_all('tr'):
    header_cells = row.find_all('th')
    for cell in header_cells:
        name = extract_column_from_header(cell)
        if name:  # Only append non-empty results
            column_names.append(name)
    

In [74]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


#### Create a dataframe by parsing the HTML laucnh tables

In [76]:
launch_dict= dict.fromkeys(column_names)

# Let's initialize the launch_dict with each value to be an empty list

for name in column_names:
    launch_dict[name] = []

# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

# Remove an irrelvant column
del launch_dict['Date and time ( )']

In [77]:
launch_dict

{'Flight No.': [],
 'Launch site': [],
 'Payload': [],
 'Payload mass': [],
 'Orbit': [],
 'Customer': [],
 'Launch outcome': [],
 'Version Booster': [],
 'Booster landing': [],
 'Date': [],
 'Time': []}

In [78]:
# This code parses the entries in the launch table, and adds it to launch_dict

extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            
            # Flight Number value
            launch_dict["Flight No."].append(flight_number)

            # Date and Time
            try:
                datatimelist = date_time(row[0])
                date = datatimelist[0].strip(',')
                time = datatimelist[1]
            except (IndexError, AttributeError, TypeError):
                date = ""
                time = ""
            launch_dict["Date"].append(date)
            launch_dict["Time"].append(time)
            
            # Booster Version
            try:
                bv = booster_version(row[1])
                if not bv and row[1].a:
                    bv = row[1].a.string.strip()
            except (IndexError, AttributeError):
                bv = ""
            launch_dict["Version Booster"].append(bv)
            
            # Launch Site
            try:
                launch_site = row[2].a.string.strip() if row[2].a else row[2].text.strip()
            except (IndexError, AttributeError):
                launch_site = ""
            launch_dict["Launch site"].append(launch_site)

            # Payload
            try:
                payload = row[3].a.string.strip() if row[3].a else row[3].text.strip()
            except (IndexError, AttributeError):
                payload = ""
            launch_dict["Payload"].append(payload)

            # Payload Mass
            try:
                payload_mass = get_mass(row[4])
            except (IndexError, AttributeError):
                payload_mass = None
            launch_dict["Payload mass"].append(payload_mass)

            # Orbit
            try:
                orbit = row[5].a.string.strip() if row[5].a else row[5].text.strip()
            except (IndexError, AttributeError):
                orbit = ""
            launch_dict["Orbit"].append(orbit)

            # Customer
            try:
                customer = row[6].a.string.strip() if row[6].a else row[6].text.strip()
            except (IndexError, AttributeError):
                customer = ""
            launch_dict["Customer"].append(customer)

            # Launch Outcome
            try:
                launch_outcome = list(row[7].strings)[0].strip()
            except (IndexError, AttributeError, TypeError):
                launch_outcome = ""
            launch_dict["Launch outcome"].append(launch_outcome)

            # Booster Landing
            try:
                booster_landing = landing_status(row[8])
            except (IndexError, AttributeError):
                booster_landing = ""
            launch_dict["Booster landing"].append(booster_landing)

In [79]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [84]:
df.isnull().sum()

Flight No.         0
Launch site        0
Payload            0
Payload mass       0
Orbit              0
Customer           0
Launch outcome     0
Version Booster    0
Booster landing    0
Date               0
Time               0
dtype: int64

In [85]:
df.to_csv('spacex_web_scraped.csv', index=False)